In [ ]:
import ROOT as r
from ROOT import TH1D, TFile, TChain, TCanvas, TBranchElement, TLorentzVector

import os
from array import array

luminosity_ifb = 10.

def createDirs(path):
    """
    Function to create any directories needed to store the output of a function.

    Args:
        path (str): the path of directories you want to exist

    Returns:
        void
    """
    base = path.split("/")[0]
    dirs = path.split("/")[:]
    tmp_dir = ""
    for folder in dirs:
        tmp_dir = tmp_dir + "/" + folder
        if not os.path.isdir(base+"/"+tmp_dir):
            try:
                os.mkdir(base+"/"+tmp_dir)
                print("creating: ", base+"/"+tmp_dir)
            except OSError as error:
                print(error)

# Define out input and output paths, make sure the output path exists.
ntuple_path = "./data/GamGam/"
output_path = "./histograms/GamGam/"
createDirs(output_path)


In [ ]:
def getNEvents(tree):
    """
    # TODO
    """
    nentries = tree.GetEntries()
    if (nentries==0):
        print("oh no, the files are empty or read in wrong :( ")
        sys.exit()
    return nentries

In [ ]:
def setupHist1D(file, name, nbins, xlow, xhigh, xlab):
    """
    # TODO
    """
    hist = TH1D(name, name+";"+xlab+";Events / bin", nbins, xlow, xhigh)
    # Make sure it stores sum of weights squared and sets bin errors as sqrt(sum-of-weights), correct for weighted histogram.
    hist.Sumw2()
    hist.SetDirectory(file)
    print("Registering histogram... ",name)
    return hist

In [ ]:
def initialiseSingleBranch(tree, branchname, vartype="f"):
    """
    Function to initialise a TBranch in the input TTree which has 1 entry per event (e.g. event number, cross section, event weight)

    Args:
        tree (ROOT TTree): input TTree that has the TBranch
        branchname (str): name of the TBranch
        vartype (str): type of branch (f, i, b python array types supported)
    Returns:
        vartype branch variable
    """
    if vartype not in ["i", "f", "b"]:
        raise Exception("not set up to read a TBranch of type {}, need to improve function beyond f/i/b...".format(vartype))
    tree.SetBranchStatus(branchname,1)
    variable = array(vartype,[0])
    tree.SetBranchAddress( branchname, variable)
    print(variable)
    return variable

def initialiseVectorBranch(tree, branchname, vartype="float"):
    """
    Function to initialise a TBranch in the input TTree which has multiple entries per event (e.g. a property for all the particles of a given type that were made)

    Args:
        tree (ROOT TTree): input TTree that has the TBranch
        branchname (str): name of the TBranch
        vartype (str): type of branch (float, int, bool root types supported)
    Returns:
        ROOT.Vector branch variable
    """
    if vartype not in ["int", "float", "bool"]:
        raise Exception("not set up to read a TBranch of type {}, need to improve function beyond float/int/bool...".format(vartype))
    tree.SetBranchStatus(branchname,1)
    variable = r.std.vector(vartype)()
    tree.SetBranchAddress( branchname, r.AddressOf(variable))
    return variable

In [ ]:
# now loop over each event, retrieve the needed info from the ntuple, check if the event passes our selection, and fill histograms for the output.
def eventLooper (tree, out_hists, is_data):
    """
    # TODO
    """

    nentries = getNEvents(tree)
    print("there are {} event in the TTree".format(nentries))

    photon_pt = initialiseVectorBranch(tree, "photon_pt", "float")
    photon_E = initialiseVectorBranch(tree, "photon_E", "float")
    photon_eta = initialiseVectorBranch(tree, "photon_eta", "float")
    photon_phi = initialiseVectorBranch(tree, "photon_phi", "float")

    mc_weight = initialiseSingleBranch(tree, "mcWeight", "f")
    xsec_ipb = initialiseSingleBranch(tree, "XSection", "f")
    sum_weights = initialiseSingleBranch(tree, "SumWeights", "f")

    SF_pileup = initialiseSingleBranch(tree, "scaleFactor_PILEUP", "f")
    # TODO include the scaleFactor_PHOTON in the event weight

    hist_pTGam_1 = setupHist1D(out_hists, "photon_pT_1", 100, 0., 500., "pT [GeV]")
    hist_pTGam_2 = setupHist1D(out_hists, "photon_pT_2", 100, 0., 500., "pT [GeV]")
    hist_EGam_1 = setupHist1D(out_hists, "photon_E_1", 100, 0., 500., "E [GeV]")
    hist_EGam_2 = setupHist1D(out_hists, "photon_E_2", 100, 0., 500., "E [GeV]")
    hist_etaGam_1 = setupHist1D(out_hists, "photon_eta_1", 10, -2.5, 2.5, "#eta")
    hist_etaGam_2 = setupHist1D(out_hists, "photon_eta_2", 10, -2.5, 2.5, "#eta")
    hist_phiGam_1 = setupHist1D(out_hists, "photon_phi_1", 10, -4., 4., "#phi")
    hist_phiGam_2 = setupHist1D(out_hists, "photon_phi_2", 10, -4., 4., "#phi")
    hist_mGamGam = setupHist1D(out_hists, "diphoton_mass", 100, 0., 1000., "m#gamma#gamma [GeV]")

    for entry in range(1, nentries):
#         # some printout to track progress
        if (entry%5000 == 0):
            pcnt_done = 100.*float(entry)/float(nentries)
            print("Processed {0} events, {1:.0f}% done.".format(entry, pcnt_done))

#         # read the entry from the ntuple
        tree.GetEntry(entry)

        # read in the event weight
        if (is_data):
            histoweight = 1.0
        else:
            # MC event weighting to luminosity of data
            histoweight = mc_weight[0] * xsec_ipb[0]*1000. * luminosity_ifb / sum_weights[0]
            # MC weight corrections for experimental effects
            histoweight = histoweight * SF_pileup[0] # TODO multiply by the photon scale factor weight

        # Obtain the kinematic variables (note TTree is in MeV and I want GeV)
                # TODO we want to check that our photons 1/2 are ordered by their pT... can you alter the assignment here to ensure we have that?
        photon_1_pt = photon_pt[0]*0.001
        photon_2_pt = photon_pt[1]*0.001
        photon_1_E = photon_E[0]*0.001
        photon_2_E = photon_E[1]*0.001
        photon_1_eta = photon_eta[0]
        photon_2_eta = photon_eta[1]
        photon_1_phi = photon_phi[0]
        photon_2_phi = photon_phi[1]

        # Only interested in events that have at least 2 photons in.
        if not (len(photon_pt) >= 2): continue
        # Need to check the photons are in the fiducial region
        if not ((abs(photon_1_eta) < 2.37 and (abs(photon_1_eta) < 1.37 or abs(photon_1_eta) > 1.56)) and\
                (abs(photon_1_eta) < 2.37 and (abs(photon_1_eta) < 1.37 or abs(photon_1_eta) > 1.56))): continue
        


        # We need to apply the photon trigger requirements, approximated by requiring our photons to have photon 1(2) pT > 35(25) GeV
        if not (photon_1_pt > 35. and photon_2_pt > 25.): continue


        # TODO we're also only interested in the case where our two photons have passed a Tight particle ID, to reduce misreconstruction backgrounds.
        # Can you use the boolean "photon_isTightID" vector branch to require this?..


        photon_1_p4 = TLorentzVector()
        photon_1_p4.SetPtEtaPhiE(photon_1_pt, photon_1_eta, photon_1_phi, photon_1_E)
        photon_2_p4 = TLorentzVector()
        photon_2_p4.SetPtEtaPhiE(photon_2_pt, photon_2_eta, photon_2_phi, photon_2_E)

        diphoton_mass = (photon_1_p4 + photon_2_p4).M()

        # Another requiremt for good photons is a pT/diphoton mass bound
        if not (photon_1_pt/diphoton_mass > 0.35 and photon_1_pt/diphoton_mass > 0.25): continue

        # Fill the histograms in the output file with the event values.
        hist_pTGam_1.Fill(photon_1_pt, histoweight)
        hist_pTGam_2.Fill(photon_2_pt, histoweight)
        hist_EGam_1.Fill(photon_1_E, histoweight)
        hist_EGam_2.Fill(photon_2_E, histoweight)
        hist_etaGam_1.Fill(photon_1_eta, histoweight)
        hist_etaGam_2.Fill(photon_2_eta, histoweight)
        hist_phiGam_1.Fill(photon_1_phi, histoweight)
        hist_phiGam_2.Fill(photon_2_phi, histoweight)
        hist_mGamGam.Fill(diphoton_mass, histoweight)


In [ ]:
# EventLoop analysis over the Data 

# write the file we will output our data histograms into
out_hists_data = TFile.Open(output_path+"data.root", "RECREATE")

# read in our data ntuples
tree_data = TChain("mini")
datafiles = [ntuple_path+"/Data/data_"+i+".GamGam.root" for i in ["A", "B", "C", "D"]]
for filename in datafiles:
    tree_data.Add(filename)

# loop over the data ntuple to process needed info into histograms
eventLooper(tree_data, out_hists_data, True)
# write data histograms to root file for further analysis.
out_hists_data.Write()
out_hists_data.Close()

In [ ]:
# EventLoop analysis over the signal MC ntuples gluon gluon fusion -> H -> gamma gamma

# write the file we will output our Higgs signal MC histograms into
out_hists_ggfHiggs = TFile.Open(output_path+"ggfHiggs.root", "RECREATE")

# read in our signal MC ntuples gluon gluon fusion -> H -> gamma gamma
tree_ggfMC = TChain("mini")
tree_ggfMC.Add(ntuple_path+"/MC/mc_343981.ggH125_gamgam.GamGam.root")

# loop over the data ntuple to process needed info into histograms
eventLooper(tree_ggfMC, out_hists_ggfHiggs, False)
# write data histograms to root file for further analysis.
out_hists_ggfHiggs.Write()
out_hists_ggfHiggs.Close()

In [ ]:
# EventLoop analysis over the signal MC ntuples Vector-boson fusion -> H -> gamma gamma

# write the file we will output our Higgs signal MC histograms into
out_hists_vbfHiggs = TFile.Open(output_path+"VBFHiggs.root", "RECREATE")

# read in our signal MC ntuples gluon gluon fusion -> H -> gamma gamma
tree_vbfMC = TChain("mini")
tree_vbfMC.Add(ntuple_path+"/MC/mc_345041.VBFH125_gamgam.GamGam.root")

# loop over the data ntuple to process needed info into histograms
eventLooper(tree_vbfMC, out_hists_vbfHiggs, False)
# write data histograms to root file for further analysis.
out_hists_vbfHiggs.Write()
out_hists_vbfHiggs.Close()

In [ ]:
# Now maybe we want to also have combined Signal MC Histograms containing both ggf and vbf production? can combine root files via hadd command:
# and we can run a bash command via python as so....
command = "hadd -f histograms/GamGam/allHiggs.root histograms/GamGam/ggfHiggs.root histograms/GamGam/VBFHiggs.root"
print(command)
os.system(command)